In [ ]:
! pip install -q kaggle
from google.colab import files
files.upload()


In [ ]:
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json



In [ ]:
 ! kaggle datasets list


In [ ]:
!kaggle datasets download -d notshrirang/spotify-million-song-dataset


Dataset URL: https://www.kaggle.com/datasets/notshrirang/spotify-million-song-dataset
License(s): CC0-1.0
 48% 10.0M/20.7M [00:00<00:00, 40.0MB/s]
100% 20.7M/20.7M [00:00<00:00, 63.8MB/s]


In [ ]:
# Unzip the downloaded file
!unzip spotify-million-song-dataset.zip

# Load the dataset into a DataFrame (replace 'filename.csv' with the actual CSV name)
import pandas as pd
data = pd.read_csv("spotify_millsongdata.csv")


Archive:  spotify-million-song-dataset.zip
  inflating: spotify_millsongdata.csv  


In [ ]:
pip install pandas scikit-learn nltk

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
import string

# Download necessary NLP tools
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

##LOADING AND PREPROCESSING THE DATA

In [ ]:
# Load the dataset (update the path to your local file)
data = pd.read_csv('/content/spotify_millsongdata.csv')

# Function to preprocess text (lowercase, remove punctuation)
def preprocess_text(text):
    text = text.lower()
    text = "".join([char for char in text if char not in string.punctuation])
    return text

# Apply preprocessing to the lyrics (text column)
data['processed_text'] = data['text'].apply(preprocess_text)

##VECTORIZE THE LYRICS

In [ ]:
# Vectorize the text using TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['processed_text'])

##COSINE SIMILARITY

In [ ]:
# Reduce the dataset size for demonstration by selecting a subset of 2000 songs
subset_data = data.sample(n=2000, random_state=42).reset_index(drop=True)

# Re-vectorize the text for the subset using TF-IDF
subset_tfidf_matrix = tfidf.fit_transform(subset_data['processed_text'])

# Recompute cosine similarity for the subset
subset_cosine_sim = cosine_similarity(subset_tfidf_matrix, subset_tfidf_matrix)


##RECOMMENDATION FUNCTION

In [ ]:
# Function to recommend songs based on cosine similarity
def recommend_songs(song_title, cosine_sim_matrix, data, num_recommendations=5):
    # Find the index of the song in the subset dataset
    indices = pd.Series(data.index, index=data['song']).drop_duplicates()

    if song_title not in indices:
        return f"Song '{song_title}' not found in the dataset."

    idx = indices[song_title]

    # Get the pairwise similarity scores for the song
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))

    # Sort the songs based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the indices of the most similar songs (excluding the song itself)
    sim_scores = sim_scores[1:num_recommendations+1]

    # Get the song indices and titles
    song_indices = [i[0] for i in sim_scores]
    return data.iloc[song_indices][['artist', 'song']]


In [ ]:
# Select a random song from the subset
sample_song = subset_data['song'].iloc[8]


In [ ]:
# Choose a specific song title from the subset
sample_song = 'Dancing Queen'  # Replace with the actual title

In [ ]:
print(f"Recommendations for '{sample_song}':")

Recommendations for 'Dancing Queen':


##TESTING

In [ ]:
# Test the recommendation system with a sample song from the subset
sample_song = subset_data['song'].iloc[0]
recommendations = recommend_songs(sample_song, subset_cosine_sim, subset_data)

# Display the recommendations
print(f"Recommendations for '{sample_song}':")
print(recommendations)


Recommendations for 'Right Or Wrong':
                 artist                       song
430            Old 97's               St. Ignatius
1973  Dusty Springfield     I've Been Wrong Before
999          Air Supply        I Don't Believe You
1508      Grateful Dead  Man Smart - Woman Smarter
564      Lynyrd Skynyrd              One More Time


In [ ]:
# Test the recommendation system with a sample song from the subset and get 10 recommendations
sample_song = subset_data['song'].iloc[0]  # Selecting the first song from the subset
recommendations = recommend_songs(sample_song, subset_cosine_sim, subset_data, num_recommendations=10)

# Display the recommendations
print(f"Recommendations for '{sample_song}':")
print(recommendations)

Recommendations for 'Right Or Wrong':
                 artist                            song
430            Old 97's                    St. Ignatius
1973  Dusty Springfield          I've Been Wrong Before
999          Air Supply             I Don't Believe You
1508      Grateful Dead       Man Smart - Woman Smarter
564      Lynyrd Skynyrd                   One More Time
866         John Legend                      Number One
71         Indigo Girls      Pushing The Needle Too Far
1403      Reba Mcentire  If I Had Any Sense Left At All
1688          Bob Seger                       Which Way
951          Zayn Malik                        It's You


In [ ]:
# Install streamlit and pyngrok
!pip install streamlit
!pip install pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.3 MB/s eta 0:00:00


In [ ]:
%%writefile streamlit_app.py

import streamlit as st
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

# Preprocess text to remove punctuation and lowercase it
def preprocess_text(text):
    text = text.lower()
    text = "".join([char for char in text if char not in string.punctuation])
    return text

# Function to get song recommendations based on cosine similarity
def recommend_songs(song_title, cosine_sim_matrix, data, num_recommendations=5):
    indices = pd.Series(data.index, index=data['song']).drop_duplicates()
    if song_title not in indices:
        return f"Song '{song_title}' not found in the dataset."

    idx = indices[song_title]
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num_recommendations+1]  # Skip the first entry (itself)
    song_indices = [i[0] for i in sim_scores]
    return data.iloc[song_indices][['artist', 'song']]

# Load the dataset
@st.cache_data
def load_data():
    data = pd.read_csv("/content/spotify_millsongdata.csv")  # Ensure this file is uploaded
    data['processed_text'] = data['text'].apply(preprocess_text)  # Preprocess the text column
    return data

# Load the dataset once
data = load_data()

# Compute TF-IDF matrix
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['processed_text'])

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Streamlit app interface
st.title("Song Recommendation System")

# Song search input
song_title = st.text_input("Enter a song title for recommendations")

if st.button("Get Recommendations"):
    if song_title:
        recommendations = recommend_songs(song_title, cosine_sim, data)
        st.write(f"Recommendations for '{song_title}':")
        st.write(recommendations)
    else:
        st.write("Please enter a song title.")


Writing streamlit_app.py


In [ ]:
!ngrok config add-authtoken 2n1MfV12T8mzWcGCBoSouFrYGuc_37Yd3feX8BM1Za8XaDvTG


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
pip install pyngrok

In [ ]:
pip install ngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 15.2 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade pyngrok
import subprocess

# Run the Streamlit app in the background
streamlit_process = subprocess.Popen(['streamlit', 'run', 'streamlit_app.py'])

# Import ngrok here so it's accessible in this cell
from pyngrok import ngrok

# Create a public URL using ngrok
public_url = ngrok.connect(8501)  # Use port 8501
print(f"Streamlit app is live at: {public_url}")


Streamlit app is live at: NgrokTunnel: "https://d39c-34-90-151-238.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!pip install --upgrade pyngrok






In [ ]:
%%writefile streamlit_app.py
import streamlit as st
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

def preprocess_text(text):
    text = text.lower()
    text = "".join([char for char in text if char not in string.punctuation])
    return text

def recommend_songs(song_title, cosine_sim_matrix, data, num_recommendations=5):
    indices = pd.Series(data.index, index=data['song']).drop_duplicates()
    if song_title not in indices:
        return f"Song '{song_title}' not found in the dataset."

    idx = indices[song_title]
    sim_scores = list(enumerate(cosine_sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:num_recommendations+1]  # Skip the first entry (itself)
    song_indices = [i[0] for i in sim_scores]
    return data.iloc[song_indices][['artist', 'song']]

@st.cache_data
def load_data():
    data = pd.read_csv("/content/spotify_millsongdata.csv")
    data['processed_text'] = data['text'].apply(preprocess_text)
    return data

data = load_data()
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['processed_text'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

st.title("Song Recommendation System")
song_title = st.text_input("Enter a song title for recommendations")
if st.button("Get Recommendations"):
    if song_title:
        recommendations = recommend_songs(song_title, cosine_sim, data)
        st.write(f"Recommendations for '{song_title}':")
        st.write(recommendations)
    else:
        st.write("Please enter a song title.")

# Step 3: Start Ngrok and Streamlit
from pyngrok import ngrok
import subprocess

# Stop any existing ngrok processes
ngrok.kill()



Overwriting streamlit_app.py


In [ ]:
import subprocess

In [ ]:
# Run the Streamlit app in the background
streamlit_process = subprocess.Popen(['streamlit', 'run', 'streamlit_app.py'])

# Create a public URL using ngrok
public_url = ngrok.connect(8501)  # Use port 8501
print(f"Streamlit app is live at: {public_url}")



Streamlit app is live at: NgrokTunnel: "https://ac9d-34-90-151-238.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
import streamlit as st
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Cache the dataset loading function
@st.cache
def load_data():
    data = pd.read_csv('spotify_millsongdata.csv')
    return data

# Cache the TF-IDF vectorizer and cosine similarity calculations
@st.cache
def process_data(data):
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(data['processed_text'])
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim

st.title('Song Recommendation System')

# Load data
data = load_data()

# Display a subset of the data
st.write(data.head(10))

# Process data only when necessary (lazy loading)
if st.button('Calculate Similarity'):
    cosine_sim = process_data(data)
    st.write("Similarity Calculated!")

2024-10-10 14:38:18.067 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-10 14:38:18.197 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-10-10 14:38:18.204 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-10 14:38:18.206 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new
caching commands, `st.cache_data` or `st.cache_resource`. More information
[in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching
logic used by `st.cache_data` and `st.cache_resource`. This might lead to some problems
or unexpected behavior in certain edge cases.

2024-10-10 14:38:18.211 No runtime found, using MemoryCacheStorageManager
2024-10-10 14:38:18.213 Thread 'MainThread': missing Sc

In [ ]:
!python streamlit_app.py


2024-10-10 14:38:32.547 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2024-10-10 14:38:32.555 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-10 14:38:32.755 
  command:

    streamlit run streamlit_app.py [ARGUMENTS]
2024-10-10 14:38:32.755 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-10 14:38:32.755 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-10 14:38:32.755 No runtime found, using MemoryCacheStorageManager
2024-10-10 14:38:33.266 Thread 'Thread-1': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-10 14:38:33.267 Thread 'Thread-1': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-10 14:38:53.160 Thr

Traceback (most recent call last):
  File "/content/streamlit_app.py", line 33, in <module>
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/pairwise.py", line 1687, in cosine_similarity
    K = safe_sparse_dot(X_normalized, Y_normalized.T, dense_output=dense_output)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py", line 205, in safe_sparse_dot
    ret = a @ b
  File "/usr/local/lib/python3.10/dist-packages/scipy/sparse/_base.py", line 695, in __matmul__
    return self._matmul_dispatch(other)
  File "/usr/local/lib/python3.10/dist-packages/scipy/sparse/_base.py", line 606, in _matmul_dispatch
    return self._matmul_sparse(other)
  File "/usr/local/lib/python3.10/dist-packages/scipy/sparse/_compressed.py", line 520, in _matmul_sparse
    nnz = fn(

In [ ]:
!streamlit run streamlit_app.py





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.196.143.23:8501

